In [59]:
import glob
import librosa
import soundfile as sf
import numpy as np
import json
import os
import tqdm

In [60]:
SOURCE_DIR = "../datasets/guitar_chords_v2/preprocessed_original"
OUTPUT_DIR = "../datasets/guitar_chords_v2/preprocessed_original_folds"

In [61]:
OUTPUT_AUDIO_DIR = os.path.join(OUTPUT_DIR, 'audio_files')

if not os.path.exists(OUTPUT_AUDIO_DIR): 
    os.makedirs(OUTPUT_AUDIO_DIR)

In [62]:
audio_files = sorted(glob.glob(os.path.join(SOURCE_DIR, 'audio_files/*.wav')))

In [63]:
# spliting dataset in 5-folds
fold_k_test  = {k:audio_files[k*64:(k+1)*64] for k in range(5)}
fold_k_train = {k:sorted(set(audio_files) - set(fold_k_test[k])) for k in range(5)} 

# validating lists
check_train, check_test = [], []
for k in range(5):
    assert set(fold_k_train[k] + fold_k_test[k]) == set(audio_files)
    check_test += fold_k_test[k]
assert set(check_test) == set(audio_files)

# exporting files to json
train_json = os.path.join(OUTPUT_DIR, 'train_folds.json')
test_json  = os.path.join(OUTPUT_DIR, 'test_folds.json')

with open(train_json, 'w') as f:
    json.dump(fold_k_train, f)
    
with open(test_json, 'w') as f:
    json.dump(fold_k_test, f)

In [64]:
# concatenate audios from a given path list and exports to an output path
def concatenate_audio_files(file_paths, output_file):
    audio_data = []

    for file_path in file_paths:
        audio, sr = librosa.load(file_path, sr=None)
        audio_data.append(audio)

    # Concatenate audio clips along the time axis (axis=0)
    concatenated_audio = np.concatenate(audio_data, axis=0)
    
    # Export the concatenated audio to a new file
    sf.write(output_file, concatenated_audio, 16000, 'PCM_16')

In [65]:
for k in tqdm.tqdm(range(5)):
    train_files = fold_k_train[k]
    train_out   = os.path.join(OUTPUT_AUDIO_DIR, f"train_fold_{k}.wav")
    concatenate_audio_files(train_files, train_out)
    
    test_files = fold_k_test[k]
    test_out   = os.path.join(OUTPUT_AUDIO_DIR, f"test_fold_{k}.wav")
    concatenate_audio_files(test_files, test_out)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.91it/s]
